## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Port Blair,11.6667,92.7500,84.04,73,89,12.91,IN,overcast clouds
1,1,Punta Arenas,-53.1500,-70.9167,50.11,71,75,11.50,CL,broken clouds
2,2,Airai,-8.9266,125.4092,80.67,43,89,2.46,TL,overcast clouds
3,3,Qaanaaq,77.4840,-69.3632,14.83,78,71,7.52,GL,broken clouds
4,4,Hithadhoo,-0.6000,73.0833,83.32,73,93,13.29,MV,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
5,5,Salalah,17.0151,54.0924,78.89,65,20,4.61,OM,few clouds
7,7,Kibala,9.1104,18.3463,75.15,88,0,0.00,TD,clear sky
9,9,Mar Del Plata,-38.0023,-57.5575,63.88,89,0,5.99,AR,clear sky
12,12,Busselton,-33.6500,115.3333,60.04,50,62,7.31,AU,broken clouds
13,13,Rikitea,-23.1203,-134.9692,75.52,81,45,16.73,PF,scattered clouds
15,15,Moron,-34.6534,-58.6198,78.03,57,25,7.00,AR,scattered clouds
16,16,San Patricio,28.0170,-97.5169,73.83,31,0,7.99,US,clear sky
18,18,Saint-Philippe,-21.3585,55.7679,72.48,86,48,7.81,RE,moderate rain
19,19,Chuy,-33.6971,-53.4616,62.87,94,0,8.41,UY,light rain
23,23,Broken Hill,-31.9500,141.4333,71.31,33,90,14.97,AU,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                258
City                   258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Country                256
Current Description    258
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                256
City                   256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Country                256
Current Description    256
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Salalah,OM,78.89,few clouds,17.0151,54.0924,
7,Kibala,TD,75.15,clear sky,9.1104,18.3463,
9,Mar Del Plata,AR,63.88,clear sky,-38.0023,-57.5575,
12,Busselton,AU,60.04,broken clouds,-33.6500,115.3333,
13,Rikitea,PF,75.52,scattered clouds,-23.1203,-134.9692,
15,Moron,AR,78.03,scattered clouds,-34.6534,-58.6198,
16,San Patricio,US,73.83,clear sky,28.0170,-97.5169,
18,Saint-Philippe,RE,72.48,moderate rain,-21.3585,55.7679,
19,Chuy,UY,62.87,light rain,-33.6971,-53.4616,
23,Broken Hill,AU,71.31,overcast clouds,-31.9500,141.4333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

# --- peeled this code off of https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
# ---

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# itinerary_df = clean_hotel_df.loc((clean_hotel_df["Hotel Name"] == "Tibati Hotel") | (clean_hotel_df["Hotel Name"] == "Quinta de Meia Eira"))
#hotel1 = "Tibati Hotel"
#hotel2 = "Quinta de Meia Eira"
#itinerary_df = clean_hotel_df.loc[(clean_hotel_df["Hotel Name"] == "Tibati Hotel") | (clean_hotel_df["Hotel Name"] == "Quinta da Meia Eira")]
#clean_hotel_df = clean_hotel_df.set_index(["Hotel Name"])
#clean_hotel_df
#itinerary_df = clean_hotel_df.loc[clean_hotel_df["Hotel Name"] == hotel1 | hotel2]
#itinerary_df = clean_hotel_df.loc["Tibati Hotel", "Quinta da Meia Eira"]
#itinerary_df